In [1]:
import os
import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans
from scipy.stats import linregress

import sys
sys.path.append('..')

import modules.plotting as pl
import modules.general as gen
import modules.signals as sig
import modules.gait_metrics as gm
import modules.assign_sides as asi
import modules.sliding_window as sw
import modules.linear_algebra as lin

In [2]:
%matplotlib auto

Using matplotlib backend: MacOSX


In [3]:
file_name = '2014-12-08_P004_Post_004'

In [4]:
best_pos_dir = os.path.join('..', 'data', 'kinect', 'best pos')
hypo_dir = os.path.join('..', 'data', 'kinect', 'processed', 'hypothesis')

hypo_paths = glob.glob(os.path.join(hypo_dir, '*.pkl'))
best_pos_paths = glob.glob(os.path.join(best_pos_dir, '*.pkl'))

hypo_paths = [x for x in hypo_paths if file_name in x]
best_pos_paths = [x for x in best_pos_paths if file_name in x]

df_hypo = pd.read_pickle(hypo_paths[0])
df_head_feet = pd.read_pickle(best_pos_paths[0])

## Split trial into separate passes

In [5]:
# Convert all position vectors to float type
# so they can be easily input to linear algebra functions
df_head_feet = df_head_feet.applymap(pd.to_numeric)

# Cluster frames with k means to locate the 4 walking passes
frames = df_head_feet.index.values.reshape(-1, 1)
k_means = KMeans(n_clusters=4, random_state=0).fit(frames)

# Sort labels so that the frames are in temporal order
labels = gen.map_sort(k_means.labels_)

# DataFrames for each walking pass in a trial
pass_dfs = gen.group_by_label(df_head_feet, labels)

In [6]:
df_pass = list(pass_dfs)[0]

In [7]:
frames = df_pass.index.values

In [8]:
_, direction_pass = gm.direction_of_pass(df_pass)

# Assign correct sides to feet
df_assigned = asi.assign_sides_pass(df_pass, direction_pass)

## Foot signal

In [9]:
signal_l = gm.foot_signal(df_assigned.L_FOOT, df_assigned.R_FOOT, direction_pass)

In [10]:
foot_dist = (df_assigned.L_FOOT - df_assigned.R_FOOT).apply(np.linalg.norm)

In [11]:
x = lin.line_coordinate_system(np.array([0, 0, 0]), direction_pass, np.stack(df_assigned.L_FOOT))

step_signal_l = pd.Series(x, index=frames)

In [12]:
signal_1 = sig.normalize(signal_l)
signal_2 = 1 - signal_1

rms_1 = sig.root_mean_square(signal_1)
rms_2 = sig.root_mean_square(signal_2)

peak_frames_1 = sig.detect_peaks(signal_1, window_length=3, min_height=rms_1)
peak_frames_2 = sig.detect_peaks(signal_2, window_length=3, min_height=rms_2)

frames_interest = np.sort(np.append(peak_frames_1, peak_frames_2))

labels = gen.label_by_split(frames, frames_interest)

sub_signals = list(gen.group_by_label(step_signal_l, labels))

In [13]:
variances = [*map(np.var, sub_signals)]
variance_array = np.array(variances).reshape(-1, 1)

In [14]:
k_means = KMeans(n_clusters=2, random_state=0).fit(variance_array)

In [15]:
variance_labels = k_means.labels_

sub_signal_lengths = [*map(len, sub_signals)]
expanded_labels = [*gen.repeat_by_list(variance_labels, sub_signal_lengths)]

In [16]:
plt.figure()
pl.scatter_labels(step_signal_l.reset_index().values, expanded_labels, s=10)

In [17]:
stance_label = np.argmin(k_means.cluster_centers_)
swing_label = 1 - stance_label

In [18]:
stance_frames = frames[expanded_labels == stance_label]

In [19]:
phase_labels = [*gen.label_repeated_elements(expanded_labels)]

In [20]:
phase_dict = {stance_label: 'stance', swing_label: 'swing'}
frame_phases = gen.map_with_dict(expanded_labels, phase_dict)

In [21]:
dict_df = {'Phase': frame_phases, 'Label': phase_labels}

df_phase = pd.DataFrame(dict_df, index=frames, dtype='category')

df_phase.index.name = 'Frame'

In [22]:
df_stance = df_phase[df_phase.Phase == 'stance']
df_swing = df_phase[df_phase.Phase == 'swing']

In [23]:
stance_series = pd.Series([*gen.label_repeated_elements(df_stance.Label.values)], index=df_stance.index) 
swing_series = pd.Series([*gen.label_repeated_elements(df_swing.Label.values)], index=df_swing.index) 

In [24]:
df_phase['Number'] = pd.concat([stance_series, swing_series])